In [1]:
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp, partial, space_eval
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score

### Data Preparing

In [2]:
df = pd.read_csv('../data/Ethos_Dataset_Binary.csv', on_bad_lines='skip', sep=';')

In [3]:
df.shape

(998, 2)

In [4]:
df.head()

,comment,isHate
0,You should know women's sports are a joke,1.0
1,You look like Sloth with deeper Down’s syndrome,1.0
2,You look like Russian and speak like Indian. B...,1.0
3,"Women deserve to be abused, I guess.",1.0
4,Women are made for making babies and cooking d...,1.0


In [5]:
df.isHate = df.isHate.apply(lambda x: 1 if x > 0.5 else 0)

In [6]:
df.to_csv('../data/Ethos_Dataset_Binary.csv', index=False, sep=';')

In [7]:
df.isHate.value_counts()

0    639
1    359
Name: isHate, dtype: int64

In [8]:
df_train, df_test = train_test_split(df, stratify=df.isHate, test_size=0.3)

In [9]:
df_train.shape, df_test.shape

((698, 2), (300, 2))

### Feature extraction

In [10]:
word_vect = TfidfVectorizer(
            sublinear_tf=True,
            strip_accents='unicode',
            analyzer='word',
            stop_words='english',
            ngram_range=(1, 2),
            max_features=2000)

In [11]:
word_vect.fit(df['comment'])
train_word_features  = word_vect.transform(df_train['comment'])
test_word_features  = word_vect.transform(df_test['comment'])

In [12]:
X_train = train_word_features.tocsr()
X_test = test_word_features.tocsr()
y_train = df_train.isHate
y_test = df_test.isHate

In [13]:
X_train.shape, X_test.shape

((698, 2000), (300, 2000))

### Model creation and evaluation

In [14]:
clf = LGBMClassifier(random_state=42) 

In [15]:
clf.fit(X_train, y_train)

LGBMClassifier(random_state=42)

In [16]:
probas_train = clf.predict_proba(X_train)
probas_test = clf.predict_proba(X_test)
predicts_train = clf.predict(X_train)
predicts_test = clf.predict(X_test)

In [17]:
metrics_train = {"roc_auc": roc_auc_score(y_train, probas_train[:, 1]),
                 "precision_macro": precision_score(y_train, predicts_train, average='macro'),
                 "recall_macro": recall_score(y_train, predicts_train, average='macro'),
                 "f1_macro": f1_score(y_train, predicts_train, average='macro')
                }

metrics_test = {"roc_auc": roc_auc_score(y_test, probas_test[:, 1]),
                "precision_macro": precision_score(y_test, predicts_test, average='macro'),
                "recall_macro": recall_score(y_test, predicts_test, average='macro'),
                "f1_macro": f1_score(y_test, predicts_test, average='macro')
                }

In [18]:
metrics_before_tuning = pd.DataFrame([metrics_train, metrics_test], index=['train', 'test']).T

In [19]:
metrics_before_tuning

,train,test
roc_auc,0.774820,0.663990
precision_macro,0.759639,0.651670
recall_macro,0.695665,0.618924
f1_macro,0.706943,0.621810


### Hyperparameters optimization

In [20]:
space = {"max_depth": hp.choice('max_depth', np.arange(4, 12, 2, dtype=int)),
         "learning_rate": hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
         "feature_fraction": hp.uniform('feature_fraction', 0.5, 1),
         "num_leaves": hp.choice('num_leaves', np.arange(16, 256, 2, dtype=int))
         }


In [21]:
def objective(params):
    lgbm = LGBMClassifier(**params)
    cv = StratifiedKFold(3)
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1).mean()
    return 1 - score

In [22]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=5)

100%|███████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.63s/trial, best loss: 0.440093868603208]


In [23]:
best_params = space_eval(space, best)
best_params

{'feature_fraction': 0.9071459634034933,
 'learning_rate': 0.12698029614839498,
 'max_depth': 8,
 'num_leaves': 88}

### Model after tuning

In [24]:
clf = LGBMClassifier(**best_params)

In [25]:
clf.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.9071459634034933, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9071459634034933


LGBMClassifier(feature_fraction=0.9071459634034933,
               learning_rate=0.12698029614839498, max_depth=8, num_leaves=88)

In [26]:
probas_train = clf.predict_proba(X_train)
probas_test = clf.predict_proba(X_test)
predicts_train = clf.predict(X_train)
predicts_test = clf.predict(X_test)

In [27]:
metrics_train_after_tuning = {"roc_auc": roc_auc_score(y_train, probas_train[:, 1]),
                 "precision_macro": precision_score(y_train, predicts_train, average='macro'),
                 "recall_macro": recall_score(y_train, predicts_train, average='macro'),
                 "f1_macro": f1_score(y_train, predicts_train, average='macro')
                }

metrics_test_after_tuning = {"roc_auc": roc_auc_score(y_test, probas_test[:, 1]),
                "precision_macro": precision_score(y_test, predicts_test, average='macro'),
                "recall_macro": recall_score(y_test, predicts_test, average='macro'),
                "f1_macro": f1_score(y_test, predicts_test, average='macro')
                }

In [28]:
metrics_after_tuning = pd.DataFrame([metrics_train_after_tuning, metrics_test_after_tuning], index=['train', 'test']).T

In [29]:
metrics_after_tuning

,train,test
roc_auc,0.770480,0.658734
precision_macro,0.761986,0.662193
recall_macro,0.688187,0.622106
f1_macro,0.699049,0.624874


In [30]:
metrics_after_tuning - metrics_before_tuning

,train,test
roc_auc,-0.004341,-0.005257
precision_macro,0.002347,0.010523
recall_macro,-0.007478,0.003183
f1_macro,-0.007894,0.003064
